# 📋**Лабораторна робота No2** 
## **Наука про дані: підготовчий етап**
### 🙆‍♀️Крупка Анна ФБ-35


**📌Мета роботи:** ознайомитися з основними кроками по роботі з даними – workflow
від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі
та природу даних, над якими виконується аналітичні операції

**📌Основні поняття:** сирі дані (raw data), підготовка даних (data preparation)

### 🟣Setup
Запустіть наступну клітинку, щоб імпортувати та налаштувати бібліотеки Python, необхідні для виконання вправи.

In [5]:
from datetime import datetime
import requests
import os
import pandas as pd
from IPython.display import display
print("Setup done")

Setup done


### 🟣Оновлення для завантаження даних
Наступний код завантажить дані для кожного регіону та збереже їх у відповідні *CSV файли*.

In [6]:
def download_function(reg_id):
    url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=" + str(reg_id) + "&year1=1981&year2=2020&type=Mean"
    timestamp = datetime.now()
    timestamp = timestamp.strftime("%Y-%m-%d_%H-%M-%S")
    folder = "Pros_data"

    filename = os.path.join(folder, f"data_region_{reg_id}_{timestamp}.csv")

    try:
        recieved_data = requests.get(url, timeout=3)
        recieved_data.raise_for_status()

        with open(filename, "w", encoding="utf-8") as file:
            file.write(recieved_data.text)

        print(f"Saving area {reg_id} in {filename} OK")

    except:
        print(f"Error in area {reg_id}")

if __name__ == "__main__":
    folder = "Pros_data"
    os.makedirs(folder, exist_ok=True)

    for reg_id in range(1, 26):
        download_function(reg_id)

Saving area 1 in Pros_data\data_region_1_2025-04-22_15-37-04.csv OK
Saving area 2 in Pros_data\data_region_2_2025-04-22_15-37-07.csv OK
Saving area 3 in Pros_data\data_region_3_2025-04-22_15-37-08.csv OK
Saving area 4 in Pros_data\data_region_4_2025-04-22_15-37-09.csv OK
Saving area 5 in Pros_data\data_region_5_2025-04-22_15-37-10.csv OK
Saving area 6 in Pros_data\data_region_6_2025-04-22_15-37-11.csv OK
Saving area 7 in Pros_data\data_region_7_2025-04-22_15-37-12.csv OK
Saving area 8 in Pros_data\data_region_8_2025-04-22_15-37-13.csv OK
Saving area 9 in Pros_data\data_region_9_2025-04-22_15-37-14.csv OK
Saving area 10 in Pros_data\data_region_10_2025-04-22_15-37-14.csv OK
Saving area 11 in Pros_data\data_region_11_2025-04-22_15-37-15.csv OK
Saving area 12 in Pros_data\data_region_12_2025-04-22_15-37-16.csv OK
Saving area 13 in Pros_data\data_region_13_2025-04-22_15-37-17.csv OK
Saving area 14 in Pros_data\data_region_14_2025-04-22_15-37-18.csv OK
Saving area 15 in Pros_data\data_regio

### 🟣Оновлення даних
Наступна функція *remake_data(directory)* зчитує всі CSV файли в директорії, очищує та об'єднує їх в один DataFrame.

In [7]:
def remake_data(directory):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_files = os.listdir(directory)
    merged = pd.DataFrame()

    for filename in all_files:
        filepath = os.path.join(directory, filename)

        try:
            df = pd.read_csv(filepath, header=1, names=headers)

            df = df[df["VHI"] != -1]

            area = filename.split("_")[2]
            df["Аrea"] = pd.to_numeric(area, errors="coerce")

            df["Year"] = df["Year"].astype(str).str.replace("<tt><pre>", "", regex=False)
            df = df[~df["Year"].str.contains('</pre></tt>', na=False)]
            df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

            df.drop('empty', axis=1, inplace=True, errors="ignore")

            merged = pd.concat([merged, df], ignore_index=True)

        except Exception as erro:
            print(f"Error in file {filename}: {erro}")

    merged.drop_duplicates(inplace=True)
    merged["Аrea"] = merged["Аrea"].astype(int)
    merged = merged.sort_values(by=["Аrea", "Year", "Week"]).reset_index(drop=True)

    return merged

df = remake_data("Pros_data")
print(df.head())
print(df.tail())
display(df)


   Year  Week    SMN     SMT    VCI    TCI    VHI  Аrea
0  1982   1.0  0.053  260.31  45.01  39.46  42.23     1
1  1982   2.0  0.054  262.29  46.83  31.75  39.29     1
2  1982   3.0  0.055  263.82  48.13  27.24  37.68     1
3  1982   4.0  0.053  265.33  46.09  23.91  35.00     1
4  1982   5.0  0.050  265.66  41.46  26.65  34.06     1
       Year  Week    SMN     SMT    VCI    TCI    VHI  Аrea
49445  2020  48.0  0.139  272.46  62.49  26.15  44.32    25
49446  2020  49.0  0.125  270.68  60.09  27.23  43.66    25
49447  2020  50.0  0.113  269.47  56.87  24.97  40.92    25
49448  2020  51.0  0.104  268.52  56.18  22.78  39.48    25
49449  2020  52.0  0.102  268.03  59.77  22.05  40.91    25


,Year,Week,SMN,SMT,VCI,TCI,VHI,Аrea
0,1982,1.0,0.053,260.31,45.01,39.46,42.23,1
1,1982,2.0,0.054,262.29,46.83,31.75,39.29,1
2,1982,3.0,0.055,263.82,48.13,27.24,37.68,1
3,1982,4.0,0.053,265.33,46.09,23.91,35.00,1
4,1982,5.0,0.050,265.66,41.46,26.65,34.06,1
...,...,...,...,...,...,...,...,...
49445,2020,48.0,0.139,272.46,62.49,26.15,44.32,25
49446,2020,49.0,0.125,270.68,60.09,27.23,43.66,25
49447,2020,50.0,0.113,269.47,56.87,24.97,40.92,25
49448,2020,51.0,0.104,268.52,56.18,22.78,39.48,25


### 🟣 Заміна індексів на назви областей  
Функція *change_area_index(df)* замінює числові індекси регіонів у DataFrame на відповідні назви областей України.


In [8]:
def change_area_index(df):
    area_index = {
        1: 'Вінницька облать', 2: 'Волинська область', 3: 'Дніпропетровська область', 4: 'Донецька область', 5: 'Житомирська область', 6: 'Закарпатська область', 7: 'Запорізька область',
        8: 'Івано-Франківська область', 9: 'Київська область', 10: 'Кіровоградська область', 11: 'Луганська область', 12: 'Львівська область', 13: 'Миколаївська область', 14: 'Одеська область',
        15: 'Полтавська область', 16: 'Рівненська область', 17: 'Сумська область', 18: 'Тернопільська область', 19: 'Харківська область', 20: 'Херсонська область', 21: 'Хмельницька область',
        22: 'Черкаська область', 23: 'Чернівецька область', 24: 'Чернігівська область', 25: 'Крим'
    }
    df["Аrea"] = df["Аrea"].replace(area_index)
    return df

df = change_area_index(df)
print(df.head())
print(df.tail())

   Year  Week    SMN     SMT    VCI    TCI    VHI              Аrea
0  1982   1.0  0.053  260.31  45.01  39.46  42.23  Вінницька облать
1  1982   2.0  0.054  262.29  46.83  31.75  39.29  Вінницька облать
2  1982   3.0  0.055  263.82  48.13  27.24  37.68  Вінницька облать
3  1982   4.0  0.053  265.33  46.09  23.91  35.00  Вінницька облать
4  1982   5.0  0.050  265.66  41.46  26.65  34.06  Вінницька облать
       Year  Week    SMN     SMT    VCI    TCI    VHI  Аrea
49445  2020  48.0  0.139  272.46  62.49  26.15  44.32  Крим
49446  2020  49.0  0.125  270.68  60.09  27.23  43.66  Крим
49447  2020  50.0  0.113  269.47  56.87  24.97  40.92  Крим
49448  2020  51.0  0.104  268.52  56.18  22.78  39.48  Крим
49449  2020  52.0  0.102  268.03  59.77  22.05  40.91  Крим


### 🟣 Перегляд VHI за регіоном і роком  
Функція *found_vhi(df, area, year)* виводить значення індексу VHI для вибраного регіону та року у вигляді таблиці.


In [9]:
def found_vhi(df, area, year):

    filtered_data = df[
        (df["Аrea"] == area) &
        (df["Year"] == year)
        ]
    selected_columns = filtered_data[['Year', 'Week', 'VHI', 'Аrea']]

    print(f"Дані VHI для {area} {year} року:")
    print(selected_columns)

area = "Сумська область"
year = 2007
found_vhi(df, area, year)

Дані VHI для Сумська область 2007 року:
       Year  Week    VHI             Аrea
32898  2007   1.0  47.47  Сумська область
32899  2007   2.0  48.33  Сумська область
32900  2007   3.0  48.76  Сумська область
32901  2007   4.0  48.62  Сумська область
32902  2007   5.0  48.09  Сумська область
32903  2007   6.0  47.48  Сумська область
32904  2007   7.0  46.54  Сумська область
32905  2007   8.0  45.64  Сумська область
32906  2007   9.0  45.33  Сумська область
32907  2007  10.0  44.24  Сумська область
32908  2007  11.0  43.07  Сумська область
32909  2007  12.0  43.59  Сумська область
32910  2007  13.0  44.02  Сумська область
32911  2007  14.0  43.51  Сумська область
32912  2007  15.0  43.99  Сумська область
32913  2007  16.0  44.31  Сумська область
32914  2007  17.0  42.79  Сумська область
32915  2007  18.0  40.93  Сумська область
32916  2007  19.0  37.71  Сумська область
32917  2007  20.0  34.34  Сумська область
32918  2007  21.0  30.03  Сумська область
32919  2007  22.0  24.91  Сумська об

### 🟣 Статистика VHI за регіоном і роком  
Функція *search_vhi(df, area, year)* обчислює та виводить мінімальне, максимальне, середнє та медіанне значення індексу VHI для вибраного регіону та року.

In [10]:
def found_vhi(df, area, year):

    filtered_data = df[
        (df["Аrea"] == area) &
        (df["Year"] == year)
        ]

    vhi_data = filtered_data["VHI"]
    vhi_min = vhi_data.min()
    vhi_max = vhi_data.max()
    vhi_avg = vhi_data.mean()
    vhi_median = vhi_data.median()

    print(f"Дані VHI для {area} {year} року:")
    print(f"Мінімальне: {vhi_min}")
    print(f"Максимальне: {vhi_max}")
    print(f"Середнє: {vhi_avg:.2f}")
    print(f"Медіанне: {vhi_median:.2f}")

area = "Сумська область"
year = 2007

found_vhi(df, area, year)

Дані VHI для Сумська область 2007 року:
Мінімальне: 5.52
Максимальне: 56.82
Середнє: 36.38
Медіанне: 43.29


### 🟣 Вибірка VHI за кількома регіонами та роками  
Функція *range_vhi(df, areas, from_y, to_y)* фільтрує дані VHI за заданим списком регіонів і діапазоном років, виводячи відповідні значення.


In [11]:
def range_vhi(df, areas, from_y, to_y):

    filtered_data = df[
        (df["Аrea"].isin(areas)) &
        (df["Year"].between(from_y, to_y))
        ]

    selected_columns = filtered_data[['Year', 'Week', 'VHI', 'Аrea']]
    print(f"Дані VHI:")
    print(selected_columns)

    return selected_columns

areas = ["Львівська область", "Сумська область"]
from_y = 1978
to_y = 2007

display(range_vhi(df, areas, from_y, to_y))

Дані VHI:
       Year  Week    VHI               Аrea
21758  1982   1.0  44.85  Львівська область
21759  1982   2.0  40.95  Львівська область
21760  1982   3.0  39.52  Львівська область
21761  1982   4.0  39.39  Львівська область
21762  1982   5.0  41.34  Львівська область
...     ...   ...    ...                ...
32945  2007  48.0  56.62    Сумська область
32946  2007  49.0  52.06    Сумська область
32947  2007  50.0  46.57    Сумська область
32948  2007  51.0  42.20    Сумська область
32949  2007  52.0  38.30    Сумська область

[2604 rows x 4 columns]


,Year,Week,VHI,Аrea
21758,1982,1.0,44.85,Львівська область
21759,1982,2.0,40.95,Львівська область
21760,1982,3.0,39.52,Львівська область
21761,1982,4.0,39.39,Львівська область
21762,1982,5.0,41.34,Львівська область
...,...,...,...,...
32945,2007,48.0,56.62,Сумська область
32946,2007,49.0,52.06,Сумська область
32947,2007,50.0,46.57,Сумська область
32948,2007,51.0,42.20,Сумська область


### 🟣 Виявлення років з екстремальною засухою  
Функція *extr_drought(df, pros=20)* знаходить роки, коли більше ніж заданий відсоток регіонів мали значення VHI нижче за критичний рівень (15), що свідчить про екстремальну засуху.


In [12]:
def extr_drought(df, pros=20):
    extr_vhi = 15
    total_areas = df["Аrea"].nunique()
    extr_pros = total_areas * (pros / 100)

    drought_years  = df[df['VHI'] < extr_vhi].groupby('Year')['Аrea'].nunique()
    critical_years = drought_years [drought_years  > extr_pros]

    print( critical_years)

    return_df = pd.DataFrame()

    for year in  critical_years.index:
        concat_df = df[(df["Year"] == year) & (df["VHI"] < extr_vhi)][["Year", "Аrea", "VHI"]]
        return_df = pd.concat([return_df, concat_df], ignore_index=True).drop_duplicates()

    return return_df

print(extr_drought(df))

Year
2000    6
Name: Аrea, dtype: int64
    Year                       Аrea    VHI
0   2000           Вінницька облать  14.64
1   2000           Вінницька облать  11.82
2   2000           Вінницька облать  10.81
3   2000           Вінницька облать  10.68
4   2000           Вінницька облать  12.30
5   2000           Вінницька облать  14.24
6   2000  Івано-Франківська область  14.61
7   2000  Івано-Франківська область  11.33
8   2000  Івано-Франківська область   9.36
9   2000  Івано-Франківська область   9.45
10  2000  Івано-Франківська область   9.73
11  2000  Івано-Франківська область  11.45
12  2000  Івано-Франківська область  14.29
13  2000          Луганська область  12.51
14  2000          Луганська область  10.60
15  2000          Луганська область  11.20
16  2000          Луганська область  12.32
17  2000          Луганська область  14.65
18  2000          Львівська область  14.89
19  2000          Львівська область  12.76
20  2000          Львівська область   7.81
21  2000      